In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 绘制模型表现图
def performance(y_true , predict , color = "g" , ann = True):
    acc = accuracy_score(y_true , predict[:] > 0.5)
    auc = roc_auc_score(y_true , predict[:])
    fpr , tpr , thr = roc_curve(y_true , predict[:])
    plt.figure()
    plt.plot(fpr , tpr )

# 读取数据

In [14]:
df_train = pd.read_csv("simplifyweibo_4_moods_training.csv",sep=',')
df_validation = pd.read_csv("simplifyweibo_4_moods_validation.csv",sep=',')
train_X = df_train['content']
train_Y = df_train['label']
print(train_Y)
validate_X = df_validation['content']
validate_Y = df_validation['label']
print(validate_X)

0       0
1       3
2       0
3       0
4       0
5       2
6       1
7       1
8       1
9       0
10      3
11      0
12      0
13      2
14      0
15      1
16      3
17      0
18      1
19      3
20      3
21      0
22      0
23      2
24      0
25      3
26      0
27      0
28      0
29      1
       ..
6370    0
6371    0
6372    0
6373    0
6374    1
6375    1
6376    1
6377    1
6378    3
6379    0
6380    3
6381    0
6382    3
6383    0
6384    0
6385    1
6386    0
6387    1
6388    0
6389    0
6390    0
6391    0
6392    2
6393    3
6394    0
6395    1
6396    1
6397    0
6398    0
6399    0
Name: label, Length: 6400, dtype: int64
0      椰丝！别和小人过不去，因为他本来就过不去；别和社会过不去，因为你会过不去；别和自己过不去，因...
1                               【爆料】这人父母怎么给他起的名字？囧 。。。。。
2                         你是风儿.....我是沙.....紫薇，我是尔康，我爸是李刚
3                        享受在宿舍听着歌看着书本的那一刻宁静。大家这时候都在干什么呢？
4      确实强悍!!【大开眼界】这个视频真给力~ 新加坡“南洋理工华乐团”演奏的。音乐响起的那一刻，...
5      一口烟进入肺的过程...仅仅30秒，刺激你的视觉。。。当你吸烟的时候，你和你身边的人的肺就变...
6      早

# 预处理

In [15]:
import jieba
import string
def clean_CN(corpus):
    stop = []
    with open(r"D:\大创项目\LDA\stopwords\CNstopwords.txt", 'r', encoding='utf-8') as f:
        for lines in f:
            stop.append(lines.strip())
    stop = set(stop)
   
    exclude = set(string.punctuation)  # 标点符号
    clean_corpus = []
    for doc in corpus:
        words = jieba.lcut(doc)
        stop_free = [i for i in words if (i not in stop) & (i.isalpha())]
        clean_corpus.append(''.join(stop_free))
    return clean_corpus

In [16]:
train_X = clean_CN(train_X)
validate_X = clean_CN(validate_X)

In [17]:
print(train_X)

['找朱桢丹丹担任司仪美容划不上号怪怪的搞娘舅节目牛尔各大颁奖典礼现场必到人物貌似不入流典礼走过场端奖杯来张照片', '超级小伪娘伪娘娃娃抓起', '缺钙长大缺爱姥姥不疼舅舅不爱左脸欠抽右脸欠踹驴驴踢猪见猪踩天生属黄瓜欠拍后天属核桃欠捶终生属破摩托欠踹', '小米晒奖品速度很快支持IPS硬屏呦IPS硬屏寄来背包收到不错背包很大结实谢谢咯', '走走忘记走想着想着忘记想看着看着忘记感觉一种迷失当初执拗固执Sowhat', '感冒心疼感冒麻陪Live感冒骚虾依然好听', '最难追处女座每日星座时间第一名处女座第二名天蝎座第三名摩羯座第四名双子座第五名水瓶座第六名狮子座第七名金牛座第八名射手座第九名天秤座第十名巨蟹座第十一名白羊座第十二名双鱼座垫底Fay', '激死一早翻屋企开电脑唔网重启次唔希望听日翻喇听日放假', '米貌帮帮唱砸活动比赛溜天终于偷得浮生日闲混沌里早中晚睡三回期间穿插修图若干最该睡觉倍儿精神放儿子美图一张勾引瞌睡虫', '太可爱猴子看着猴猴拼命表情身体狗狗无助神态真的难过人类冷漠感到自责大鸟大鸟泪奔难过震撼與人之間一丁點話大愛南京爆炸现场可爱猴子抱狗狗逃命可爱人性', '祝演唱会顺利表紧张口号期待恐惧失望', '史上聪明宣传人员中华情亲情中华欢聚多伦多官方宣传片秒版本正式选用周笔畅最新单曲MISSMISSINGME背景音乐敬请期待', '恩跟着走主席代言人世界青年人朝气蓬勃清晨九点钟太阳希望寄托身上', '三尺之上神明', '我乐霸道', '喜欢女人姜叔滴耳朵型一点点罗拔迪尼路味道妈呀口水牛芭沙拍提前曝光恐吓力爱极品男人子弹飞', '何梅梅好想你星期不见想', '饭店吃味道不错吐什么世道操吃鸽子小心咯老鼠肉饭店酒席椒盐鸽子肉转给', '得罪湖南村书记铁链活活锁住骨头日前湖南省张家界市永定区官黎坪办事处赶子坪村位于张家界市区城郊一户好心人家收留湖南省怀化市沅陵县借母溪乡流浪逃命而来母子儿子岁老家堂姐发生口角纠纷村支书村主任村妇女主任联合非法拘禁多年', '疯天天骗面对真实情况接受坚强新闻报报喜报忧希望能量战胜宇宙俞妈妈入院天才进重症房探班想象需承受痛苦宝贝永远爱灏明宝贝持续坚强加油', '回复成灰泪始干抢沙发站共产主义对立面喜欢中国文化喜欢中国共产主义想中国经济增长社会成熟度越来越高人会共产主义看法石原慎太郎接受南方人物周刊时采访表述', '強那句经典gd

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf_extractor(corpus,ngram_range=(1,1)):
    vectorizer = TfidfVectorizer(min_df=3,norm='l2',smooth_idf=True,use_idf=True,ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer,features

tfidf_vectorizer,tfidf_train_features = tfidf_extractor(train_X)
tfidf_validate_features = tfidf_vectorizer.transform(validate_X)

In [24]:
tfidf_test_features.shape

(800, 1)

词向量模型训练完毕


# 模型训练和评估

In [41]:
# 作出混淆矩阵
from sklearn import metrics
def get_metrics(true_labels, predicted_labels):
    print('Accuracy' + str(np.round(metrics.accuracy_score(true_labels, predicted_labels), 2)))
    print('Percision' + str(np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 2)))
    print('Recall' + str(np.round(metrics.recall_score(true_labels, predicted_labels, average='weighted'), 2)))
    print('F1 Score' + str(np.round(metrics.f1_score(true_labels, predicted_labels, average='weighted'), 2)))


# # 定义函数使用机器学习算法训练模型


def train_predict_evaluate_model(classifier, train_features, train_labels, test_features, test_labels):
    # 使用分类器训练数据
    model = classifier.fit(train_features, train_labels)
    # 使用训练好的模型对测试集进行预测
    predictions = classifier.predict(test_features)
    # 对模型表现进行评估
    get_metrics(true_labels=test_labels, predicted_labels=predictions)
    return predictions,model


from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', n_iter_no_change=30)
svm_avgwv_predictions,svm_argwv_model=train_predict_evaluate_model(svm,avg_wv_train_features,train_Y,avg_wv_test_features,validate_Y)
report = classification_report(svm_avgwv_predictions,validate_Y)
print(report)

Accuracy0.55
Percision0.38
Recall0.55
F1 Score0.4
              precision    recall  f1-score   support

           0       0.98      0.55      0.71       780
           1       0.01      0.33      0.02         3
           2       0.00      0.00      0.00         2
           3       0.03      0.20      0.05        15

    accuracy                           0.55       800
   macro avg       0.25      0.27      0.19       800
weighted avg       0.96      0.55      0.69       800



In [49]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0,n_estimators=100,max_depth=10)
rfc_avgwv_predictions,rfc_argwv_model=train_predict_evaluate_model(rfc,avg_wv_train_features,train_Y,avg_wv_test_features,validate_Y)
report = classification_report(rfc_avgwv_predictions,validate_Y)
print(report)


Accuracy0.56
Percision0.52
Recall0.56
F1 Score0.41
              precision    recall  f1-score   support

           0       0.99      0.56      0.72       781
           1       0.01      0.50      0.02         2
           2       0.02      0.50      0.03         4
           3       0.04      0.38      0.08        13

    accuracy                           0.56       800
   macro avg       0.27      0.49      0.21       800
weighted avg       0.97      0.56      0.70       800



In [53]:
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=15,
                      learning_rate=0.1,
                      n_estimators=2000,
                      min_child_weight=5,
                      max_delta_step=0,
                      subsample=0.8,
                      colsample_bytree=0.7,
                      reg_alpha=0,
                      reg_lambda=0.4,
                      scale_pos_weight=0.8,
                      silent=True,
                      objective='binary:logistic',
                      missing=None,
                      eval_metric='auc',
                      seed=1440,
                      gamma=0)
xgb_avgwv_predictions,xgb_argwv_model=train_predict_evaluate_model(xgb,avg_wv_train_features,train_Y,avg_wv_test_features,validate_Y)
report = classification_report(xgb_avgwv_predictions,validate_Y)
print(report)


Accuracy0.52
Percision0.43
Recall0.52
F1 Score0.44
              precision    recall  f1-score   support

           0       0.88      0.58      0.70       667
           1       0.07      0.24      0.10        34
           2       0.07      0.26      0.11        34
           3       0.14      0.25      0.18        65

    accuracy                           0.52       800
   macro avg       0.29      0.33      0.27       800
weighted avg       0.75      0.52      0.60       800



In [ ]:
df_test = pd.read_csv("simplifyweibo_4_moods_test.csv",sep=',')
test_X = df_train['content']
test_X = clean_CN(test_X)
avg_wv_test_features = averaged_word_vectorizer(corpus=test_X, model=model, num_features=100)
predictions = svm_tfidf_model.predict(avg_wv_test_features).tolist()
print(predictions)
